In [2]:
import torchvision.models as models
import torchvision.ops as ops
import torch
from torch import nn
from typing import OrderedDict

In [52]:
img = torch.zeros(1,3,128,1024)

def forward_first_layer(model, image):
        """Executing forward pass for the zeroth Retina Net layer"""
        x = model.conv1(image)
        x = model.bn1(x)
        x = model.relu(x)
        x = model.maxpool(x)
        return x

class Layer(nn.Sequential):
    def __init__(self,in_channels,out_channels):
        # [1, 512, 4, 32]
        super().__init__(
            nn.ReLU(),
            #[64, 128, 256, 512, 64, 64],
            #i= 512, o=64
            #i=64, o=64
            # next
            # i=64, o=64
            # i=64, o=64
            nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=1, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=2, padding=0),
            nn.ReLU(),
        )




In [1]:
#out_channels = [64, 128, 256, 512, 64, 64] # without fpn
out_channels = [256, 256, 256, 256, 512 , 256]
out_channels = [64 , 128, 256, 512, 256, 128]
feature_sizes=[[32, 256], [16, 128], [8, 64], [4, 32], [2, 16], [1, 8]],

model = models.resnet34(pretrained=True)
layer5 = Layer(512, 256)
layer6 = Layer(256, 128)

fpn = ops.FeaturePyramidNetwork([64 , 128, 256, 512, 256, 128], 256)
oout_features = []
features_dict = OrderedDict()

# Layer 0
x = forward_first_layer(model,img)
print('layer 0: ', x.shape)

# Layer 1
features_dict['feat0'] = model.layer1(x)
print('feat0: ', features_dict['feat0'].shape)

# Layer 2
features_dict['feat1'] = model.layer2(features_dict['feat0'])
print('feat1: ', features_dict['feat1'].shape)

# Layer 3
features_dict['feat2'] = model.layer3(features_dict['feat1'])
print('feat2: ', features_dict['feat2'].shape)

# Layer 4
features_dict['feat3'] = model.layer4(features_dict['feat2'])
print('feat3: ', features_dict['feat3'].shape)

#img3 = torch.zeros([1,512,4,32])
# Layer 5
features_dict['feat4'] = layer5(features_dict['feat3'])
print('feat4: ', features_dict['feat4'].shape)

# Layer 6
features_dict['feat5'] = layer6(features_dict['feat4'])
print('feat5: ', features_dict['feat5'].shape)
# Forward to FPN
out_features = fpn(features_dict)



NameError: name 'models' is not defined